# Actividad: Regresión Líneal Tokio

## Carga de librerías y dataframe

In [2725]:
#Cargamos librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2726]:
# Cargar archivo CSV usando pandas
df = pd.read_csv('Tokio_Limpio.csv')
df.head(2)

,Unnamed: 0.1,last_scraped,name,description,neighborhood_overview,host_location,host_response_time,host_is_superhost,host_verifications,host_has_profile_pic,...,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,review_scores_rating,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month
0,0,2024-09-27,Oshiage Holiday Apartment,?,?,"Tokyo, Japan",within a day,t,"['email', 'phone']",t,...,22.3,7.0,1.0,4.78,4.91,4.84,4.83,4.55,4.80,1.14
1,1,2024-09-27,Kero-kero house room 1,We have been in airbnb since 2011 and it has g...,We love Nishinippori because is nearer to Toky...,Japan,within an hour,t,"['email', 'phone']",t,...,22.3,10.0,2.0,4.98,4.91,4.98,4.98,4.84,4.92,1.79


## Creación de los dataframes con las variables a utilizar

In [2727]:
# Convertimos las variables price y review_scores_rating a numérico
df['price'] = pd.to_numeric(df['price'].astype(str).str.replace('[$,]', '', regex=True), errors='coerce')
df['review_scores_rating'] = pd.to_numeric(df['review_scores_rating'].astype(str).str.replace('[$,]', '', regex=True), errors='coerce')

In [2728]:
# Rellenamos los valores NaN con bfill
df['price'].fillna(method='bfill', inplace=True)


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_3508\3229511809.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['price'].fillna(method='bfill', inplace=True)
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_3508\3229511809.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['price'].fillna(method='bfill', inplace=True)


In [2729]:
# Cambiamos los valores '?' el valor que mas se repite en la columna (t o f) para la columna host_is_superhost
df['host_is_superhost'] = df['host_is_superhost'].replace({'?': df['host_is_superhost'].mode()[0]})
df['has_availability'] = df['has_availability'].replace({'?': df['has_availability'].mode()[0]})

In [2730]:
# Creamos una copia del DataFrame para las variables dependientes: host_is_superhost, host_has_profile_pic, host_identity_verified, has_availability, instant_bookable, price, property_type, accommodates, room_type, review_scores_rating.
df_Dep = df[['host_is_superhost', 'host_has_profile_pic', 
             'host_identity_verified', 'has_availability', 
             'instant_bookable', 'price', 'property_type', 
             'accommodates', 'room_type', 'review_scores_rating']].copy()
df_Dep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18736 entries, 0 to 18735
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   host_is_superhost       18736 non-null  object 
 1   host_has_profile_pic    18736 non-null  object 
 2   host_identity_verified  18736 non-null  object 
 3   has_availability        18736 non-null  object 
 4   instant_bookable        18736 non-null  object 
 5   price                   18736 non-null  float64
 6   property_type           18736 non-null  object 
 7   accommodates            18736 non-null  float64
 8   room_type               18736 non-null  object 
 9   review_scores_rating    18736 non-null  float64
dtypes: float64(3), object(7)
memory usage: 1.4+ MB


In [2731]:
# Convertimos instant_bookable a binario
df['instant_bookable'] = df['instant_bookable'].map({'t': 1, 'f': 0})

In [2732]:
# Creamos una copia del DataFrame para las variables independientes: host_response_rate, number_of_reviews, review_scores_rating, review_scores_communication, availability_365, minimum_nights, maximum_nights, availability_30, accommodates, bedrooms, bathrooms, price, review_scores_cleanliness, review_scores_value, instant_bookable.
df_Ind = df[['host_response_rate', 'number_of_reviews',
             'review_scores_rating', 'review_scores_communication', 
             'availability_365', 'minimum_nights', 'maximum_nights', 
             'availability_30', 'accommodates', 'bedrooms', 
             'bathrooms', 'beds', 'price', 'review_scores_cleanliness', 
             'review_scores_value', 'instant_bookable']].copy()
df_Ind.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18736 entries, 0 to 18735
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   host_response_rate           18736 non-null  float64
 1   number_of_reviews            18736 non-null  float64
 2   review_scores_rating         18736 non-null  float64
 3   review_scores_communication  18736 non-null  float64
 4   availability_365             18736 non-null  int64  
 5   minimum_nights               18736 non-null  float64
 6   maximum_nights               18736 non-null  float64
 7   availability_30              18736 non-null  float64
 8   accommodates                 18736 non-null  float64
 9   bedrooms                     18736 non-null  float64
 10  bathrooms                    18736 non-null  float64
 11  beds                         18736 non-null  float64
 12  price                        18736 non-null  float64
 13  review_scores_cl

## Convertir a dicótomicas algunas variables dependientes

In [2733]:
# Convertimos la variable price a dicotómica a partir de su promedio 
df_Dep['price'] = df_Dep['price'].apply(lambda x: 1 if x > df_Dep['price'].mean() else 0)
df_Dep['price'] = df_Dep['price'].replace({1: 'higher', 0: 'lower'})
df_Dep['price'].value_counts()

price
lower     12872
higher     5864
Name: count, dtype: int64

In [2734]:
# Convertimos la variable review_scores_rating a dicotómica a partir de su promedio
df_Dep['review_scores_rating'] = df_Dep['review_scores_rating'].apply(lambda x: 1 if x > df_Dep['review_scores_rating'].mean() else 0)
df_Dep['review_scores_rating'] = df_Dep['review_scores_rating'].replace({1: 'higher', 0: 'lower'}) 
df_Dep['review_scores_rating'].value_counts()

review_scores_rating
higher    10985
lower      7751
Name: count, dtype: int64

In [2735]:
# Convertimos la variable accommodates a dicotómica a partir de su promedio
df_Dep['accommodates'] = df_Dep['accommodates'].apply(lambda x: 1 if x > df_Dep['accommodates'].mean() else 0)
df_Dep['accommodates'] = df_Dep['accommodates'].replace({1: 'higher', 0: 'lower'})
df_Dep['accommodates'].value_counts()

accommodates
lower     12896
higher     5840
Name: count, dtype: int64

In [2736]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['property_type'])
df_Dep['property_type'].value_counts()

property_type
Entire rental unit             11288
Entire home                     2382
Room in hotel                    882
Private room in home             730
Private room in rental unit      684
                               ...  
Private room in barn               1
Entire bungalow                    1
Treehouse                          1
Private room in camper/rv          1
Shipping container                 1
Name: count, Length: 64, dtype: int64

In [2737]:
# Convertimos la variable property_type a dicotómica con Entire rental unit y el resto other
df_Dep['property_type'] = df_Dep['property_type'].apply(lambda x: 1 if x == 'Entire rental unit' else 0)
df_Dep['property_type'] = df_Dep['property_type'].replace({1: 'entire', 0: 'other'})
df_Dep['property_type'].value_counts()

property_type
entire    11288
other      7448
Name: count, dtype: int64

In [2738]:
#Verificar los valores sin repetirse de una columna
unico = np.unique(df['room_type'])
df_Dep['room_type'].value_counts()

room_type
Entire home/apt    15109
Private room        3058
Shared room          424
Hotel room           145
Name: count, dtype: int64

In [2739]:
# Convertimos la variable room_type a dicotómica con Entire rental unit y el resto other
df_Dep['room_type'] = df_Dep['room_type'].apply(lambda x: 1 if x == 'Entire home/apt' else 0)
df_Dep['room_type'] = df_Dep['room_type'].replace({1: 'apt', 0: 'other'})
df_Dep['room_type'].value_counts()

room_type
apt      15109
other     3627
Name: count, dtype: int64

## Regresión líneal de las variables

### host_is_superhost

In [2740]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['host_response_rate', 'number_of_reviews', 'review_scores_rating']]
Vars_Dep = df_Dep['host_is_superhost']

In [2741]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [2742]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [2743]:
#se escalan todos los datos
escalar = StandardScaler()

In [2744]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [2745]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [2746]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [2747]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['f', 'f', 'f', ..., 'f', 'f', 't'], dtype=object)

In [2748]:
unico = np.unique(df_Dep['host_is_superhost'])
unico

array(['f', 't'], dtype=object)

In [2749]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[3233  270]
 [1460  658]]


In [2750]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.709051724137931


In [2751]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.6922255826365415


In [2752]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.9229232086782757


### host_has_profile_pic

In [2753]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['host_response_rate', 'number_of_reviews', 'review_scores_communication']]
Vars_Dep = df_Dep['host_has_profile_pic']

In [2754]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [2755]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [2756]:
#se escalan todos los datos
escalar = StandardScaler()

In [2757]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [2758]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [2759]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [2760]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [2761]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0   67]
 [   0 5554]]


In [2762]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.988080412737947


In [2763]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.988080412737947


In [2764]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.0


### host_identity_verified

In [2765]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['host_response_rate', 'number_of_reviews', 'availability_365']]
Vars_Dep = df_Dep['host_identity_verified']

In [2766]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [2767]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [2768]:
#se escalan todos los datos
escalar = StandardScaler()

In [2769]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [2770]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [2771]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [2772]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [2773]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0  172]
 [   0 5449]]


In [2774]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.9694004625511475


In [2775]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.9694004625511475


In [2776]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.0


### has_availability

In [2777]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['minimum_nights', 'maximum_nights', 'availability_30', 'availability_365']]
Vars_Dep = df_Dep['has_availability']

In [2778]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [2779]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [2780]:
#se escalan todos los datos
escalar = StandardScaler()

In [2781]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [2782]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [2783]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [2784]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [2785]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0    2]
 [   0 5619]]


In [2786]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.9996441914250134


In [2787]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.9996441914250134


In [2788]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.0


### instant_bookable

In [2789]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['host_response_rate', 'number_of_reviews', 'review_scores_rating']]
Vars_Dep = df_Dep['instant_bookable']

In [2790]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [2791]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [2792]:
#se escalan todos los datos
escalar = StandardScaler()

In [2793]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [2794]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [2795]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [2796]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['t', 't', 't', ..., 't', 't', 't'], dtype=object)

In [2797]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0 1404]
 [   0 4217]]


In [2798]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="t")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.7502223803593666


In [2799]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.7502223803593666


In [2800]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="f")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.0


### price

In [2801]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['accommodates', 'bedrooms', 'bathrooms', 'number_of_reviews', 'review_scores_rating']]
Vars_Dep = df_Dep['price']

In [2802]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [2803]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [2804]:
#se escalan todos los datos
escalar = StandardScaler()

In [2805]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [2806]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [2807]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [2808]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['lower', 'lower', 'lower', ..., 'higher', 'lower', 'lower'],
      dtype=object)

In [2809]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[ 779  957]
 [ 320 3565]]


In [2810]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="higher")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.7088262056414922


In [2811]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.7728162248710194


In [2812]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="lower")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.9176319176319176


### property_type

In [2813]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['accommodates', 'bedrooms', 'price', 'review_scores_rating']]
Vars_Dep = df_Dep['property_type']

In [2814]:
# Verificamos que las variables independientes no tengan valores nulos
print(Vars_Indep.isnull().sum())

accommodates            0
bedrooms                0
price                   0
review_scores_rating    0
dtype: int64


In [2815]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [2816]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [2817]:
#se escalan todos los datos
escalar = StandardScaler()

In [2818]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [2819]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [2820]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [2821]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['entire', 'entire', 'entire', ..., 'entire', 'entire', 'entire'],
      dtype=object)

In [2822]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[2867  519]
 [1606  629]]


In [2823]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="entire")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.6409568522244579


In [2824]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.6219533890766767


In [2825]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="other")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.2814317673378076


### accommodates

In [2826]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['bedrooms', 'bathrooms', 'beds', 'price', 'number_of_reviews']]
Vars_Dep = df_Dep['accommodates']

In [2827]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [2828]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [2829]:
#se escalan todos los datos
escalar = StandardScaler()

In [2830]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [2831]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [2832]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [2833]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['higher', 'lower', 'higher', ..., 'lower', 'lower', 'higher'],
      dtype=object)

In [2834]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[1149  594]
 [ 305 3573]]


In [2835]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="higher")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.7902338376891335


In [2836]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.8400640455434976


In [2837]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="lower")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.9213512119649304


### room_type

In [2838]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['accommodates', 'bedrooms', 'price', 'instant_bookable']]
Vars_Dep = df_Dep['room_type']

In [2839]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [2840]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [2841]:
#se escalan todos los datos
escalar = StandardScaler()

In [2842]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [2843]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [2844]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [2845]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['apt', 'apt', 'apt', ..., 'apt', 'apt', 'apt'], dtype=object)

In [2846]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[4552   18]
 [ 979   72]]


In [2847]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="apt")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.8229976496112819


In [2848]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.8226294253691514


In [2849]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="other")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.06850618458610847


### review_scores_rating

In [2850]:
#Dec1aramos las variables dependientes e independientes para la regresión Logística
Vars_Indep = df_Ind[['number_of_reviews', 'review_scores_cleanliness', 'review_scores_communication', 'review_scores_value']]
Vars_Dep = df_Dep['review_scores_rating']

In [2851]:
#Redefinimos las variables
X = Vars_Indep
y = Vars_Dep

In [2852]:
#Dividimos eI conjunto de datos en la parte de entrenamiento y prueba:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

In [2853]:
#se escalan todos los datos
escalar = StandardScaler()

In [2854]:
#Para realizar el escalamiento de las variables "X" tanto de entrenamiento como de prueba, utilizamos el método "fit_transform"
X_train = escalar.fit_transform(X_train)
X_test = escalar.transform(X_test)

In [2855]:
#Definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo = LogisticRegression()

In [2856]:
#Entrenamos el modelo
algoritmo.fit(X_train, y_train)

LogisticRegression()

In [2857]:
#Realizamos una predicción
y_pred = algoritmo.predict(X_test)
y_pred

array(['higher', 'higher', 'higher', ..., 'higher', 'higher', 'higher'],
      dtype=object)

In [2858]:
#Verifico la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[2854  440]
 [ 535 1792]]


In [2859]:
#Calculo la precisión del modelo
from sklearn.metrics import precision_score

precision = precision_score(y_test, y_pred, average="binary", pos_label="higher")
print('Precisión del modelo: ')
print(precision)

Precisión del modelo: 
0.8421363233992328


In [2860]:
#Calculo la exactitud del modelo
from sklearn.metrics import accuracy_score

exactitud = accuracy_score(y_test, y_pred)
print('Exactitud del modelo: ')
print(exactitud)

Exactitud del modelo: 
0.8265433196940046


In [2861]:
#Calculo la sensibilidad del modelo
from sklearn.metrics import recall_score

sensibilidad = recall_score(y_test, y_pred, average="binary", pos_label="lower")
print('Sensibilidad del modelo: ')
print(sensibilidad)

Sensibilidad del modelo: 
0.7700902449505801
